In [1]:


import pyspark

from pyspark.sql import SparkSession  #import sparksession from pyspark.sql


spark = SparkSession.builder.master("local[*]").appName("Test SQL app").getOrCreate()

spark = SparkSession.builder.appName('CreditCardSystem').getOrCreate()
#creat spark session

JSON Extract Function 

In [30]:
# def extract_from_json():

 #Read the Json data form my local computer 
cdw_sapp_branch_json = spark.read.json("cdw_sapp_branch.json")
cdw_sapp_credit_json = spark.read.json("cdw_sapp_credit.json")
cdw_sapp_custmer_json = spark.read.json("cdw_sapp_custmer.json")

#Show the schema of the Json data
cdw_sapp_branch_json.printSchema()
cdw_sapp_credit_json.printSchema()
cdw_sapp_custmer_json.printSchema()
print(cdw_sapp_branch_json.show())
print(cdw_sapp_credit_json.show())
    # create a list of dataframe
# extracted_data = [cdw_sapp_branch_json,cdw_sapp_credit_json,cdw_sapp_custmer_json]

    #reaturn list of extracted data
#return extracted_data 

root
 |-- BRANCH_CITY: string (nullable = true)
 |-- BRANCH_CODE: long (nullable = true)
 |-- BRANCH_NAME: string (nullable = true)
 |-- BRANCH_PHONE: string (nullable = true)
 |-- BRANCH_STATE: string (nullable = true)
 |-- BRANCH_STREET: string (nullable = true)
 |-- BRANCH_ZIP: long (nullable = true)
 |-- LAST_UPDATED: string (nullable = true)

root
 |-- BRANCH_CODE: long (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_SSN: long (nullable = true)
 |-- DAY: long (nullable = true)
 |-- MONTH: long (nullable = true)
 |-- TRANSACTION_ID: long (nullable = true)
 |-- TRANSACTION_TYPE: string (nullable = true)
 |-- TRANSACTION_VALUE: double (nullable = true)
 |-- YEAR: long (nullable = true)

root
 |-- APT_NO: string (nullable = true)
 |-- CREDIT_CARD_NO: string (nullable = true)
 |-- CUST_CITY: string (nullable = true)
 |-- CUST_COUNTRY: string (nullable = true)
 |-- CUST_EMAIL: string (nullable = true)
 |-- CUST_PHONE: long (nullable = true)
 |-- CUST_STATE: str

In [31]:

cdw_sapp_custmer_json.show()
#mapping
# convert names in title
#middle name in lower case

#addres concatinate together Street name and app no ( separteted by comma)
# Phone number based on the format

#select First_name 


+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|    Redwood Drive|
|   683|4210653310116272|     Huntley|United States| WDunham@exam

Transform 


Mapping cdw_sapp_custmer_json 

Drict Move - 

Convert -  the Name to Title Case
        - the middle name in lower case
        - the Last Name in Title Case
        - Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment)
        - Change the format of phone number to (XXX)XXX-XXXX

1, Transform data types of each data and Concatenate Apartment no and Street name of customer's Residence with comma as a seperator (Street, Apartment) to STREET_NAME,APT_NO and also Convert the Name to Title Case

APT_NO: string convert to VARCHAR
CREDIT_CARD_NO: string convert to  VARCHAR
CUST_CITY: string convert to  VARCHAR
CUST_COUNTRY: string convert to  VARCHAR
CUST_EMAIL: string convert to  VARCHAR
CUST_PHONE: long convert to  VARCHAR
CUST_STATE: string convert to  VARCHAR
CUST_ZIP: string convert to int INT
FIRST_NAME: string convert to VARCHAR
LAST_NAME: string convert to VARCHAR  
LAST_UPDATED: string convert to TIMESTAMP
MIDDLE_NAME: string convert to VARCHAR
SSN: long convert to int
STREET_NAME: string convert to VARCHAR
FULL_STREET_ADDRESS convert to VARCHAR

In [5]:
cdw_sapp_custmer_json.dtypes

[('APT_NO', 'string'),
 ('CREDIT_CARD_NO', 'string'),
 ('CUST_CITY', 'string'),
 ('CUST_COUNTRY', 'string'),
 ('CUST_EMAIL', 'string'),
 ('CUST_PHONE', 'bigint'),
 ('CUST_STATE', 'string'),
 ('CUST_ZIP', 'string'),
 ('FIRST_NAME', 'string'),
 ('LAST_NAME', 'string'),
 ('LAST_UPDATED', 'string'),
 ('MIDDLE_NAME', 'string'),
 ('SSN', 'bigint'),
 ('STREET_NAME', 'string')]

StructType([StructField('APT_NO', StringType(), True), StructField('CREDIT_CARD_NO', StringType(), True), StructField('CUST_CITY', StringType(), True), StructField('CUST_COUNTRY', StringType(), True), StructField('CUST_EMAIL', StringType(), True), StructField('CUST_PHONE', LongType(), True), StructField('CUST_STATE', StringType(), True), StructField('CUST_ZIP', StringType(), True), StructField('FIRST_NAME', StringType(), True), StructField('LAST_NAME', StringType(), True), StructField('LAST_UPDATED', StringType(), True), StructField('MIDDLE_NAME', StringType(), True), StructField('SSN', LongType(), True), StructField('STREET_NAME', StringType(), True)])

In [32]:
#mapping logic Change the format of phone number to (XXX)XXX-XXXX   CUST_PHONE


from pyspark.sql.functions import initcap, concat, lit, col, lower, substring
cdw_sapp_custmer=cdw_sapp_custmer_json.withColumn("FIRST_NAME" , initcap(col("FIRST_NAME"))  #tile case
                                                  ).withColumn("LAST_NAME" , initcap(col("LAST_NAME"))
                                                  ).withColumn("MIDDLE_NAME" , lower(col("MIDDLE_NAME"))
                                                  ).withColumn("FULL_STREET_ADDRESS" , concat(col("STREET_NAME"), lit(","), ("APT_NO"))
                                                  ).withColumn("CUST_PHONE",concat(lit('('),substring(col("CUST_PHONE"),1,3),lit(')'),
                                                                  substring(col("CUST_PHONE"),4,3),lit('-'),
                                                                  substring(col("CUST_PHONE"),7,4)))
cdw_sapp_custmer.show(5)

#df.select 



+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME| FULL_STREET_ADDRESS|
+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|(123)781-8|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North|Main Street North...|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|(123)893-3|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|    R

In [33]:
from pyspark.sql.functions import initcap, concat, lit, col

#convert last name data type to VARCAR
cdw_sapp_custmer.withColumn("LAST_NAME", col("LAST_NAME").cast("VARCHAR(20)")
                                
                                ).withColumn("MIDDLE_NAME", col("MIDDLE_NAME").cast("VARCHAR(20)")
                                ).withColumn("LAST_NAME", col("LAST_NAME").cast("VARCHAR(20)")
                                ).withColumn("APT_NO", col("APT_NO").cast("VARCHAR(20)")             
                                ).withColumn("CREDIT_CARD_NO", col("CREDIT_CARD_NO").cast("VARCHAR(20)")
                                ).withColumn("CUST_CITY", col("CUST_CITY").cast("VARCHAR(20)") 
                                ).withColumn("CUST_COUNTRY", col("CUST_COUNTRY").cast("VARCHAR(20)") 
                                ).withColumn("CUST_EMAIL", col("CUST_EMAIL").cast("VARCHAR(20)")     
                                ).withColumn("CUST_PHONE", col("CUST_PHONE").cast("VARCHAR(20)")
                                ).withColumn("CUST_STATE", col("CUST_STATE").cast("VARCHAR(20)")
                                ).withColumn("CUST_ZIP", col("APT_NO").cast("INT")
                                ).withColumn("LAST_UPDATED", col("LAST_UPDATED").cast("TIMESTAMP")
                                ).withColumn("SSN", col("SSN").cast("INT")
                                ).withColumn("STREET_NAME", col("STREET_NAME").cast("VARCHAR(20)")
                                ).withColumn("FULL_STREET_ADDRESS", col("FULL_STREET_ADDRESS").cast("VARCHAR(50)"))
                                             
                                             #convert last name data type

#Phone (xxx)xxx-xxx

DataFrame[APT_NO: string, CREDIT_CARD_NO: string, CUST_CITY: string, CUST_COUNTRY: string, CUST_EMAIL: string, CUST_PHONE: string, CUST_STATE: string, CUST_ZIP: int, FIRST_NAME: string, LAST_NAME: string, LAST_UPDATED: timestamp, MIDDLE_NAME: string, SSN: int, STREET_NAME: string, FULL_STREET_ADDRESS: string]

In [34]:
cdw_sapp_custmer.show(5)  #CUST_PHONE needs area code

+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|         CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME| FULL_STREET_ADDRESS|
+------+----------------+------------+-------------+-------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|   656|4210653310061055|     Natchez|United States|AHooper@example.com|(123)781-8|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         wm|123456100|Main Street North|Main Street North...|
|   829|4210653310102868|Wethersfield|United States|EHolman@example.com|(123)893-3|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    brendan|123453023|    R

Mapping cdw_sapp_branch_json fille 

In [35]:
cdw_sapp_branch_json.dtypes

[('BRANCH_CITY', 'string'),
 ('BRANCH_CODE', 'bigint'),
 ('BRANCH_NAME', 'string'),
 ('BRANCH_PHONE', 'string'),
 ('BRANCH_STATE', 'string'),
 ('BRANCH_STREET', 'string'),
 ('BRANCH_ZIP', 'bigint'),
 ('LAST_UPDATED', 'string')]

BRANCH_CODE into INT
BRANCH_NAME convert to VARCHAR
BRANCH_STREET convert to VARCHAR
BRANCH_CITY convert to VARCHAR
BRANCH_STATE convert to VARCHAR
BRANCH_ZIP  INT
BRANCH_PHONE convert to VARCHAR
LAST_UPDATED convert to TIMESTAMP


DataFrame[BRANCH_CITY: string, BRANCH_CODE: int, BRANCH_NAME: string, BRANCH_PHONE: string, BRANCH_STATE: string, BRANCH_STREET: string, BRANCH_ZIP: int, LAST_UPDATED: timestamp]

Mapping Logic for Branch 


BRANCH_ZIP - If the source value is null load default (99999) value else Direct move
BRANCH_PHONE - Change the format of phone number to (XXX)XXX-XXXX





In [36]:
from pyspark.sql.functions import when,col, substring, concat,lit
cdw_sapp_branch = cdw_sapp_branch_json.withColumn("BRANCH_ZIP", when(col("BRANCH_ZIP").isNull(),'99999').otherwise(col("BRANCH_ZIP"))
                                ).withColumn("BRANCH_PHONE",concat(lit('('),substring(col("BRANCH_PHONE"),1,3),lit(')'),
                                                                  substring(col("BRANCH_PHONE"),4,3),lit('-'),
                                                                  substring(col("BRANCH_PHONE"),7,4)))

In [37]:
cdw_sapp_branch.withColumn("BRANCH_CODE", col("BRANCH_CODE").cast("INT")
                               ).withColumn('BRANCH_NAME', col('BRANCH_NAME').cast("VARCHAR(20)")
                                ).withColumn('BRANCH_STREET', col('BRANCH_STREET').cast("VARCHAR(20)")
                                ).withColumn('BRANCH_CITY', col('BRANCH_CITY').cast("VARCHAR(20)") 
                                ).withColumn('BRANCH_STATE', col('BRANCH_STATE').cast("VARCHAR(20)")
                                ).withColumn('BRANCH_ZIP', col('BRANCH_CODE').cast("INT")
                                ).withColumn('BRANCH_PHONE', col('BRANCH_PHONE').cast("VARCHAR(20)")
                                ).withColumn("LAST_UPDATED", col('LAST_UPDATED').cast("TIMESTAMP"))                              
                          

DataFrame[BRANCH_CITY: string, BRANCH_CODE: int, BRANCH_NAME: string, BRANCH_PHONE: string, BRANCH_STATE: string, BRANCH_STREET: string, BRANCH_ZIP: int, LAST_UPDATED: timestamp]

In [38]:
cdw_sapp_branch.show(5)



+-----------------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
|      BRANCH_CITY|BRANCH_CODE| BRANCH_NAME| BRANCH_PHONE|BRANCH_STATE|    BRANCH_STREET|BRANCH_ZIP|        LAST_UPDATED|
+-----------------+-----------+------------+-------------+------------+-----------------+----------+--------------------+
|        Lakeville|          1|Example Bank|(123)456-5276|          MN|     Bridle Court|     55044|2018-04-18T16:51:...|
|          Huntley|          2|Example Bank|(123)461-8993|          IL|Washington Street|     60142|2018-04-18T16:51:...|
|SouthRichmondHill|          3|Example Bank|(123)498-5926|          NY|    Warren Street|     11419|2018-04-18T16:51:...|
|       Middleburg|          4|Example Bank|(123)466-3064|          FL| Cleveland Street|     32068|2018-04-18T16:51:...|
|    KingOfPrussia|          5|Example Bank|(123)484-9701|          PA|      14th Street|     19406|2018-04-18T16:51:...|
+-----------------+-----

cdw_sapp_credit.json
Mapping Logic
Convert DAY, MONTH, and YEAR into a TIMEID (YYYYMMDD)

Targeted data type
Source File Column Names
CREDIT_CARD_NO  convert in to VARCHAR
DAY, MONTH, YEAR convert in to VARCHAR
CUST_SSN convert in to INT
BRANCH_CODE convert in to INT
TRANSACTION_TYPE convert in to VARCHAR
TRANSACTION_VALUE convert in to DOUBLE
TRANSACTION_ID  convert in to INT


In [39]:
from pyspark.sql.functions import col, lpad,substring

#use lpad function to for month and day because some of them put as one degit

cdw_sapp_credit = cdw_sapp_credit_json.withColumn("TIMEID",concat(col("YEAR"),lpad(col("MONTH"),2,"0"),lpad(col("DAY"),2,"0")))
cdw_sapp_credit.show(5)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|  TIMEID|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|20180214|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|20180320|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|20180708|
|        114|4210653349028689|123459988| 19|    4|             4|   Entertainment|            59.73|2018|20180419|
|         93|4210653349028689|123459988| 10|   10|             5|             Gas|             3.59|2018|20181010|
+-----------+----------------+---------+---+-----+--------------+---------------

In [40]:
cdw_sapp_credit.show(5)

+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
|BRANCH_CODE|  CREDIT_CARD_NO| CUST_SSN|DAY|MONTH|TRANSACTION_ID|TRANSACTION_TYPE|TRANSACTION_VALUE|YEAR|  TIMEID|
+-----------+----------------+---------+---+-----+--------------+----------------+-----------------+----+--------+
|        114|4210653349028689|123459988| 14|    2|             1|       Education|             78.9|2018|20180214|
|         35|4210653349028689|123459988| 20|    3|             2|   Entertainment|            14.24|2018|20180320|
|        160|4210653349028689|123459988|  8|    7|             3|         Grocery|             56.7|2018|20180708|
|        114|4210653349028689|123459988| 19|    4|             4|   Entertainment|            59.73|2018|20180419|
|         93|4210653349028689|123459988| 10|   10|             5|             Gas|             3.59|2018|20181010|
+-----------+----------------+---------+---+-----+--------------+---------------

In [41]:
#Change CREDIT_CARD_NO column name to CUST_CC_NO and cast it VARCHAr
cdw_sapp_credit=cdw_sapp_credit.withColumn("CUST_CC_NO", col("CREDIT_CARD_NO").cast("VARCHAR(20)")
                           ).withColumn("TIMEID", col("TIMEID").cast("VARCHAR(20)")
                           ).withColumn("CUST_SSN", col("CUST_SSN").cast("INT")
                           ).withColumn("BRANCH_CODE", col("BRANCH_CODE").cast("INT")
                           ).withColumn("TRANSACTION_TYPE", col("TRANSACTION_TYPE").cast("VARCHAR(20)")
                           ).withColumn("TRANSACTION_VALUE", col("TRANSACTION_VALUE").cast("DOUBLE")
                           ).withColumn("TRANSACTION_ID", col("TRANSACTION_ID").cast("INT"))




In [42]:
df_cdw_sapp_credit= cdw_sapp_credit.select("CUST_CC_NO", "TIMEID","CUST_SSN","BRANCH_CODE","TRANSACTION_TYPE","TRANSACTION_VALUE","TRANSACTION_ID")

In [43]:
df_cdw_sapp_credit.dtypes

[('CUST_CC_NO', 'string'),
 ('TIMEID', 'string'),
 ('CUST_SSN', 'int'),
 ('BRANCH_CODE', 'int'),
 ('TRANSACTION_TYPE', 'string'),
 ('TRANSACTION_VALUE', 'double'),
 ('TRANSACTION_ID', 'int')]

In [44]:
df_cdw_sapp_custmer=cdw_sapp_custmer.select("SSN", "FIRST_NAME","MIDDLE_NAME","LAST_NAME","CREDIT_CARD_NO","FULL_STREET_ADDRESS","CUST_CITY","CUST_STATE","CUST_COUNTRY","CUST_ZIP","CUST_PHONE","CUST_EMAIL","LAST_UPDATED")
df_cdw_sapp_custmer.show(5)



+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+----------+-------------------+--------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO| FULL_STREET_ADDRESS|   CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|         CUST_EMAIL|        LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+------------+----------+-------------+--------+----------+-------------------+--------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|Main Street North...|     Natchez|        MS|United States|   39120|(123)781-8|AHooper@example.com|2018-04-21T12:49:...|
|123453023|      Etta|    brendan|   Holman|4210653310102868|   Redwood Drive,829|Wethersfield|        CT|United States|   06109|(123)893-3|EHolman@example.com|2018-04-21T12:49:...|
|123454487|    Wilber|   ezequiel|   Dunham|4210653310116272|12th Street East,683|     Hun

In [45]:
#Making ready datas to load on database
df_cdw_sapp_branch=cdw_sapp_branch.select("BRANCH_CODE","BRANCH_NAME","BRANCH_STREET","BRANCH_CITY", "BRANCH_STATE","BRANCH_ZIP","BRANCH_PHONE" ,"LAST_UPDATED")
df_cdw_sapp_branch.show(5)



+-----------+------------+-----------------+-----------------+------------+----------+-------------+--------------------+
|BRANCH_CODE| BRANCH_NAME|    BRANCH_STREET|      BRANCH_CITY|BRANCH_STATE|BRANCH_ZIP| BRANCH_PHONE|        LAST_UPDATED|
+-----------+------------+-----------------+-----------------+------------+----------+-------------+--------------------+
|          1|Example Bank|     Bridle Court|        Lakeville|          MN|     55044|(123)456-5276|2018-04-18T16:51:...|
|          2|Example Bank|Washington Street|          Huntley|          IL|     60142|(123)461-8993|2018-04-18T16:51:...|
|          3|Example Bank|    Warren Street|SouthRichmondHill|          NY|     11419|(123)498-5926|2018-04-18T16:51:...|
|          4|Example Bank| Cleveland Street|       Middleburg|          FL|     32068|(123)466-3064|2018-04-18T16:51:...|
|          5|Example Bank|      14th Street|    KingOfPrussia|          PA|     19406|(123)484-9701|2018-04-18T16:51:...|
+-----------+-----------

df_cdw_sapp_custmer
df_cdw_sapp_branch

In [ ]:
#Create a Database in SQL(MySQL), named “creditcard_capstone.”

#) Create a Python and Pyspark Program to load/write the “Credit card System Data” into RDBMS(creditcard_capstone).
# Tables should be created by the following names in RDBMS:
# CDW_SAPP_BRANCH
# CDW_SAPP_CREDIT_CARD
# CDW_SAPP_CUSTOMER


In [22]:
spark = SparkSession.builder.master("local[*]").appName("creditcard_capstone").getOrCreate()

# Write the spark DataFrame into CompanyABC_DB database 
# Use CompanyABC_DB database url and credentials
# Table name CompanyABC_stock
cdw_sapp_branch_json.write.format("jdbc") \
  .mode("append") \
  .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
  .option("dbtable", "df_cdw_sapp_branch") \
  .option("dbtable", "df_cdw_sapp_credit") \
  .option("dbtable", "df_cdw_sapp_custmer") \
  .option("user", "root") \
  .option("password", "Godislove@123") \
  .save()

In [ ]:


# df_Price.write.format("jdbc") \
#   .mode("append") \
#   .option("url", "jdbc:mysql://localhost:3306/creditcard_capstone") \
#   .option("dbtable", "CDW_SAPP_BRANCH") \
#   .option("dbtable", "DW_SAPP_CREDIT_CARD") \
#   .option("dbtable", "CDW_SAPP_CUSTOMER") \
#   .option("user", secret.mysql_username) \
#   .option("password", secrets.mysql_password) \
#   .save()

###2. Functional Requirements - Application Front-End#

Functional Requirements 2.1 

1) Used to display the transactions made by customers living in a
given zip code for a given month and year. Order by day in
descending order.
2) Used to display the number and total values of transactions for a
given type.
3) Used to display the total number and total values of transactions
for branches in a given state.

cdw_sapp_credit_json Have  CUST_SSN which is the primary key and TRANSACTION_ID, YEAR, DAY

cdw_sapp_custmer_json Have CUST SSN, CUST_ZIP


In [23]:
#def transaction()
# Get zip_code, Month, year from the user
zip_code = input("Please enter the Zip code: ")
Month = input("Please enter the month: ")
Year = input("Please enter the year")

#Split the TIMEID in to Day, Month and Year before satrt the quary 
df_cdw_sapp_credit.show(5)

#use lpad function to for month and day because some of them put as one degit

df_TIMEID = df_cdw_sapp_credit.withColumn("YEAR", substring(col("TIMEID"), 1,4).cast("VARCHAR(20)")
                                          ).withColumn("MONTH", (substring(col("TIMEID"), 5,2).cast("VARCHAR(20)"))
                                          ).withColumn("DAY", (substring(col("TIMEID"), 7,5).cast("VARCHAR(20)")))



df_TIMEID.show(5)


#transaction = cdw

+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
|4210653349028689|20180214|123459988|        114|       Education|             78.9|             1|
|4210653349028689|20180320|123459988|         35|   Entertainment|            14.24|             2|
|4210653349028689|20180708|123459988|        160|         Grocery|             56.7|             3|
|4210653349028689|20180419|123459988|        114|   Entertainment|            59.73|             4|
|4210653349028689|20181010|123459988|         93|             Gas|             3.59|             5|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+
only showing top 5 rows

+----------------+--------+---------+-----------+----------------+---------

In [ ]:
#spark = SparkSession.builder.master("local[*]").appName("Test SQL app").getOrCreate()

#Used to display the transactions made by customers living in a given zip code for a given month and year. 
#Order by day in descending order.



We hav 3 data table now we will create temporary view for each table with out modiffing the orginal file query from tep 
df_cdw_sapp_branch 
df_cdw_sapp_credit
df_CDW_SAPP_CREDIT_CARD

In [24]:
#creat temporary from the datafream df_cdw_sapp_credit
df_cdw_sapp_credit.createOrReplaceTempView("credit_card")

df_cdw_sapp_custmer.createOrReplaceTempView("Custmer")

df_cdw_sapp_branch.createOrReplaceTempView("branch")

#use spark.sql to query the view and show
lol = spark.sql("SELECT * FROM credit_Card")

#Conver TIMEID to Month and Year by spelit or using substring slicing 
df_TIMEID = lol.withColumn("YEAR", substring(col("TIMEID"), 1,4).cast("VARCHAR(20)")
                                          ).withColumn("MONTH", (substring(col("TIMEID"), 5,2).cast("VARCHAR(20)"))
                                          ).withColumn("DAY", (substring(col("TIMEID"), 7,5).cast("VARCHAR(20)")))


df_TIMEID.createOrReplaceTempView("df_TIMEID")
df_TIMEID.show(5)

# Used to display the transactions made by customers living in agiven zip code for a given month and year.
#  Order by day in descending order.

spark_sql = f""" SELECT t.*,c.* FROM df_TIMEID t INNER JOIN custmer c ON t.CUST_SSN = c.SSN WHERE c.CUST_ZIP = '{zip_code}' AND t.MONTH ='{Month}' AND t.YEAR = '{Year}' ORDER BY t.DAY DESC"""

result = spark.sql(spark_sql)    
result.show() 
          
          #SELECT * FROM df_TIMEID d INNER JOIN c on d.CREDIT_CARD_NO = c.CREDIT_CARD_NO where d.CUST_ZIP = '{zip_code}'").show()




+----------------+--------+---------+-----------+----------------+-----------------+--------------+----+-----+---+
|      CUST_CC_NO|  TIMEID| CUST_SSN|BRANCH_CODE|TRANSACTION_TYPE|TRANSACTION_VALUE|TRANSACTION_ID|YEAR|MONTH|DAY|
+----------------+--------+---------+-----------+----------------+-----------------+--------------+----+-----+---+
|4210653349028689|20180214|123459988|        114|       Education|             78.9|             1|2018|   02| 14|
|4210653349028689|20180320|123459988|         35|   Entertainment|            14.24|             2|2018|   03| 20|
|4210653349028689|20180708|123459988|        160|         Grocery|             56.7|             3|2018|   07| 08|
|4210653349028689|20180419|123459988|        114|   Entertainment|            59.73|             4|2018|   04| 19|
|4210653349028689|20181010|123459988|         93|             Gas|             3.59|             5|2018|   10| 10|
+----------------+--------+---------+-----------+----------------+--------------

In [25]:
#2) Used to display the number and total values of transactions for agiven type.

type = input("Please enter a transactions type: ")

#use count to get total number of transaction

spark_sql= f""" SELECT COUNT(*) AS TRANSATION_TYPE, '{type}' FROM df_TIMEID WHERE df_TIMEID.TRANSACTION_TYPE = '{type}'"""

result = spark.sql(spark_sql)    
result.show() 
          

+---------------+---+
|TRANSATION_TYPE|Gas|
+---------------+---+
|           6605|Gas|
+---------------+---+



In [26]:
#3) Used to display the total number and total values of transactions for branches in a given state.

State = input("Please enter the state like(WA): ")

spark_sql =f""" SELECT '{State}' AS STATE, COUNT(df_TIMEID.TRANSACTION_ID) AS TOTAL_TRANSACTION, 
SUM(df_TIMEID.TRANSACTION_VALUE) AS TOTAL_TRANSACTION_VALUE 
FROM df_TIMEID JOIN branch ON branch.BRANCH_CODE = df_TIMEID.BRANCH_CODE 
WHERE branch.BRANCH_STATE = '{State}' """


result = spark.sql(spark_sql)    
result.show() 






+-----+-----------------+-----------------------+
|STATE|TOTAL_TRANSACTION|TOTAL_TRANSACTION_VALUE|
+-----+-----------------+-----------------------+
|   WA|              833|      43439.25999999999|
+-----+-----------------+-----------------------+



2.2 Customer Details Module
Req-2.2 Customer Details
Functional
Requirements 2.2
1) Used to check the existing account details of a customer.
2) Used to modify the existing account details of a customer.
3) Used to generate a monthly bill for a credit card number for a
given month and year.
4) Used to display the transactions made by a customer between
two dates. Order by year, month, and day in descending order

In [28]:
#1) Used to check the existing account details of a customer.

SSN = input("Please enter the 9 digit of customer SSN(123456100): ")

#spark_Sqll = f""" SELECT '{SSN}', CREDIT_CARD_NO, FIRST_NAME,MIDDLE_NAME,LAST_NAME, CREDIT_CARD_NO, FULL_STREET_ADDRESS, CUST_CITY,CUST_STATE
spark_Sql = f""" SELECT * FROM Custmer WHERE '{SSN}' = SSN """
result = spark.sql(spark_Sql)
result.show()


+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+
|      SSN|FIRST_NAME|MIDDLE_NAME|LAST_NAME|  CREDIT_CARD_NO| FULL_STREET_ADDRESS|CUST_CITY|CUST_STATE| CUST_COUNTRY|CUST_ZIP|CUST_PHONE|         CUST_EMAIL|        LAST_UPDATED|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+
|123456100|      Alec|         wm|   Hooper|4210653310061055|Main Street North...|  Natchez|        MS|United States|   39120|(123)781-8|AHooper@example.com|2018-04-21T12:49:...|
+---------+----------+-----------+---------+----------------+--------------------+---------+----------+-------------+--------+----------+-------------------+--------------------+



In [28]:
#2) Used to modify the existing account details of a customer

spark = SparkSession.builder.master("local[*]").appName("creditcard_capstone").getOrCreate()

if "df_cdw_sapp_credit" in spark.catalog.listTables():

    customer_details = spark.read.table("df_cdw_sapp_custmer")
    SSN = input("Please enter the 9 digits of Customer SSN(123456000): ")


    Custumer_Info = int(input("Please enter the number you wish to change:\n1,FIRST_NAME \n2,MIDDLE_NAME \n3,LAST_NAME \n4,CREDIT_CARD_NO \n5,FULL_STREET_ADDRESS \n6,CUST_CITY \n7,CUST_STATE \n8,CUST_COUNTRY \n9,CUST_ZIP \n10, CUST_PHONE: ")) 
    update = input("Please enter new change: ")

    columns = ["FIRST_NAME","MIDDLE_NAME","LAST_NAME","CREDIT_CARD_NO","FULL_STREET_ADDRESS","CUST_CITY", "CUST_STATE","CUST_COUNTRY","CUST_ZIP","CUST_PHON"]
    column_update = columns[update -1]
else:
    print("The table df_cdw_sapp_credit does not exist")

The table df_cdw_sapp_credit does not exist


In [93]:
#2) Used to modify the existing account details of a customer
SSN = input("Please enter the 9 digits of Customer SSN(123456000): ")


Custumer_Info = int(input("Please enter the number you wish to change:\n1,FIRST_NAME \n2,MIDDLE_NAME \n3,LAST_NAME \n4,CREDIT_CARD_NO \n5,FULL_STREET_ADDRESS \n6,CUST_CITY \n7,CUST_STATE \n8,CUST_COUNTRY \n9,CUST_ZIP \n10, CUST_PHONE: "))       
              
spark_sql = f"UPDATE Custmer SET SSN = '{SSN}'"

#spark_sql = f"UPDATE Customer SET SSN ='{SSN}'"

if Custumer_Info == '1':
  FIRST_NAME = input("Please enter the new First Name: ")
  spark_sql += f", FIRST_NAME = '{FIRST_NAME}'"
elif Custumer_Info ==2:
  MIDDLE_NAME = input("Please enetr the Middle Name: ")
  spark_sql += f",MIDDLE_NAME = '{MIDDLE_NAME}'"
elif Custumer_Info ==3:
   LAST_NAME = input("Please enter the new Last Name: ") 
   spark_sql += f",LAST_NAME= '{LAST_NAME}'"
elif Custumer_Info ==4:
   CREDIT_CARD_NO = input("Please enter new CREDIT_CARD_NO:  ")
   spark_sql += f",CREDIT_CARD_NO= '{CREDIT_CARD_NO}'"

#Excute the MYSQL query
spark.sql(spark_sql)
#spark.sql(spark_sql)

Update_Custumer_info = spark.sql(f"SELECT * FROM Custmer WHERE SSN = '{SSN}'")
Update_Custumer_info.show()


Py4JJavaError: An error occurred while calling o25.sql.
: org.apache.spark.SparkUnsupportedOperationException: UPDATE TABLE is not supported temporarily.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.ddlUnsupportedTemporarilyError(QueryExecutionErrors.scala:1109)
	at org.apache.spark.sql.execution.SparkStrategies$BasicOperators$.apply(SparkStrategies.scala:896)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$1(QueryPlanner.scala:63)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.plan(QueryPlanner.scala:93)
	at org.apache.spark.sql.execution.SparkStrategies.plan(SparkStrategies.scala:70)
	at org.apache.spark.sql.catalyst.planning.QueryPlanner.$anonfun$plan$3(QueryPlanner.scala:78)
	at scala.collection.TraversableOnce$folder$1.apply(Traversab

3. Functional Requirements - Data Analysis and Visualization
After data is loaded into the database, users can make changes from the front end, and they can also
view data from the front end. Now, the business analyst team wants to analyze and visualize the data.
Use Python libraries for the below requirements:
Page 4 of 7
9/27/2023 – edited VLB
Req - 3 Data Analysis and Visualization
Functional
Requirements 3.1
Find and plot which transaction type has the highest transaction count.
Note: Save a copy of the visualization to a folder in your github,
making sure it is PROPERLY NAMED!
Functional
Requirements 3.2
Find and plot which state has a high number of customers.
Note: Save a copy of the visualization to a folder in your github,
making sure it is PROPERLY NAMED!
Functional
Requirements 3.3
Find and plot the sum of all transactions for the top 10 customers,
and which customer has the highest transaction amount.
Hint (use CUST_SSN).
Note: Save a copy of the visualization t

In [51]:
#3) Used to generate a monthly bill for a credit card number for a given month and year.

credits_Card_number = input("Please enter the credit card number(ex:4210653349028689): ")
Month = input("Please enter the month: ")
Year = input("Please enter the year: ")

#spark_Sqll = f""" SELECT '{SSN}', CREDIT_CARD_NO, FIRST_NAME,MIDDLE_NAME,LAST_NAME, CREDIT_CARD_NO, FULL_STREET_ADDRESS, CUST_CITY,CUST_STATE
spark_Sql = f""" SELECT  SUM(TRANSACTION_VALUE) AS Transaction FROM df_TIMEID , credit_card.FIRST_NAME,credit_card.MIDDLE_NAME credit_card.LAST_NAME|  CREDIT_CARD_NO 
WHERE '{credits_Card_number }' = CUST_CC_NO AND YEAR = '{Year}' AND MONTH = '{Month}'"""

Total_amount_spent = spark.sql(spark_Sql).collect()[0]["Transaction"]
print(f"Total Amount spent: {Total_amount_spent}")



Total Amount spent: 358.91


In [52]:
#4) Used to display the transactions made by a customer between two dates. Order by year, month, and day in descending order.

credits_Card_number = input("Please enter the credit card number(ex:4210653349028689): ")
print("In order to display the transactions made by a custommer needs date")
Start_Date = input("Please enter starting  day,month and year [20180210]: ")
End_Date = input("Please enter end data [20180210]")

#use credit_card 
spark_sql = f""" SELECT * FROM TIMEID WHERE CUST_CC_NO = '{credits_Card_number}' AND TIMEID >= {Start_Date} <= {End_Date }
ORDER BY TIMEID, DESC TIMEID """

result = spark.sql(spark_sql)

result.show()


In order to display the transactions made by a custommer needs date


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

Req - 3 Data Analysis and Visualization


In [53]:
#Find and plot which transaction type has the highest transaction
#count.Note: Take a screenshot of the graphs.


import pandas as pd 
import matplotlib.pyplot as plt

#From df_cdw_sapp_credit dataframe group transaction type and count the transactions
count = df_cdw_sapp_credit['TRANSACTION_TYPE'].value_counts()

#Get transaction type and highest count
top_transaction_type = count.counts.idxmax()
top_transaction_count = count.count.max()

#Plot transaction 

count.plot(kind = 'bar', title = 'Transaction Type with the Highest Transaction Count')

plt.xlable('Transaction Type')
plt.ylabel('Transaction Count')

plt.show()



ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it